In [13]:
!pip install face_recognition
!pip install numpy
!pip install ultralytics opencv-python matplotlib

  Using cached face_recognition-1.3.0-py2.py3-none-any.whl.metadata (21 kB)
  Using cached face_recognition_models-0.3.0-py2.py3-none-any.whl
  Using cached click-8.1.8-py3-none-any.whl.metadata (2.3 kB)
  Using cached dlib-19.24.6.tar.gz (3.4 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
Using cached face_recognition-1.3.0-py2.py3-none-any.whl (15 kB)
Using cached click-8.1.8-py3-none-any.whl (98 kB)
Failed to build dlib


  error: subprocess-exited-with-error
  
  × Building wheel for dlib (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [44 lines of output]
      <string>:234: SyntaxWarning: invalid escape sequence '\('
      <string>:235: SyntaxWarning: invalid escape sequence '\('
      <string>:236: SyntaxWarning: invalid escape sequence '\('
      running bdist_wheel
      running build
      running build_ext
      
      
                         CMake is not installed on your system!
      
          Or it is possible some broken copy of cmake is installed on your system.
          It is unfortunately very common for python package managers to include
          broken copies of cmake.  So if the error above this refers to some file
          path to a cmake file inside a python or anaconda or miniconda path then you
          should delete that broken copy of cmake from your computer.
      
          Instead, please get an official copy of cmake from one of these known good
   

In [ ]:
import face_recognition
import cv2
import numpy as np
import csv
import os
from datetime import datetime

# Safely access the camera
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Error: Unable to access the camera.")
    exit()

# Load images and encode
def load_and_encode(path):
    image = face_recognition.load_image_file(path)
    encoding = face_recognition.face_encodings(image)
    if encoding:
        return encoding[0]
    else:
        raise Exception(f"No face found in image: {path}")

known_face_encodings = [
    load_and_encode(r"C:\python\Student attandence\photos\stevejobs.png"), #path of the Photos
    load_and_encode(r"C:\python\Student attandence\photos\ratantata.png"),
    load_and_encode(r"C:\python\Student attandence\photos\monalisa.png"),
    load_and_encode(r"C:\python\Student attandence\photos\tesla.png")
]

known_faces_names = [
    "Steve Jobs",
    "Ratan Tata",
    "Monalisa",
    "Tesla"
]

students = known_faces_names.copy()

# Prepare CSV file for today's attendance
now = datetime.now()
current_date = now.strftime("%Y-%m-%d")
f = open(current_date + '.csv', 'w+', newline='')
lnwriter = csv.writer(f)

# Main loop
while True:
    ret, frame = cap.read()
    if not ret:
        print("Error: Unable to read from the camera.")
        break

    # Resize frame for faster recognition
    small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)
    rgb_small_frame = cv2.cvtColor(small_frame, cv2.COLOR_BGR2RGB)

    # Detect faces and compute encodings
    face_locations = face_recognition.face_locations(rgb_small_frame)
    face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)

    face_names = []
    for face_encoding in face_encodings:
        matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
        name = ""

        face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
        best_match_index = np.argmin(face_distances)

        if matches[best_match_index]:
            name = known_faces_names[best_match_index]

        face_names.append(name)

        if name in students:
            students.remove(name)
            current_time = now.strftime("%H:%M:%S")
            lnwriter.writerow([name, current_time])
            print(f"{name} marked present at {current_time}")

    # Display the frame
    for (top, right, bottom, left), name in zip(face_locations, face_names):
        # Scale back up since the frame we used was scaled down
        top *= 4
        right *= 4
        bottom *= 4
        left *= 4

        # Draw a rectangle and name
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)
        cv2.putText(frame, name, (left, top - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)

    cv2.imshow("Attendance System", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Clean up
cap.release()
cv2.destroyAllWindows()
f.close()


ModuleNotFoundError: No module named 'face_recognition'